In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting  import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

from acquire import *

from env import user,host,password

In [2]:
def time_split(df, train_size = .5, validate_size = .3):
    '''Splits time series data based on percentages and returns train, validate, test THE
    DATAFRAME MUST BE CHRONOLOGICALLY SORTED!'''
    t_size = int(len(df) * train_size)
    v_size = int(len(df) * validate_size)
    end = t_size + v_size
    return df[0:t_size], df[t_size:end], df[end:len(df)+1]

In [3]:
df = pd.read_csv('saas.csv')
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [4]:
# Fix column names
new_names = {col: col.lower() for col in df.columns}
df.rename(columns=new_names, inplace=True)
df.head()

,month_invoiced,customer_id,invoice_id,subscription_type,amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [5]:
# Convert month_invoiced to datetime object to sort properly
df.month_invoiced = pd.to_datetime(df.month_invoiced)

In [6]:
# Date range is between Jan 31st 2014 and Dec 31st 2017
print('min date', df.month_invoiced.min(), '\n', 'max date:', df.month_invoiced.max())

## Examine features
# Seems that subscription type is categorical with 5 different types (0 - 4)
print('Sub Types', df.subscription_type.value_counts())

# It appears that the amount is associated with the sub type (0, 5 ,10, 15, 20)
print('Amount counts', df.amount.value_counts())

# There are duplicate customer_id observations for certain id's aggregating based on those
# might be a good start point
df.customer_id.value_counts()

min date 2014-01-31 00:00:00 
 max date: 2017-12-31 00:00:00
Sub Types 0.0    496410
1.0    178212
2.0     70706
3.0     26474
4.0     18656
Name: subscription_type, dtype: int64
Amount counts 0.0     496410
5.0     178212
10.0     70706
15.0     26474
20.0     18656
Name: amount, dtype: int64


1001961    48
1014093    48
1012840    48
1000265    48
1008461    48
           ..
1026527     1
1029387     1
1002772     1
1029257     1
1017545     1
Name: customer_id, Length: 29400, dtype: int64

In [7]:
df.head()

,month_invoiced,customer_id,invoice_id,subscription_type,amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [15]:
# Create an ongoing_total column to showcase cummulative revenue from each customer_id
df['ongoing_total'] = df.groupby(df.customer_id)['amount'].cumsum()
df.set_index('month_invoiced')

,customer_id,invoice_id,subscription_type,amount,ongoing_total
month_invoiced,,,,,
2014-01-31,1000000,5000000,0.0,0.0,0.0
2014-01-31,1000001,5000001,2.0,10.0,10.0
2014-01-31,1000002,5000002,0.0,0.0,0.0
2014-01-31,1000003,5000003,0.0,0.0,0.0
2014-01-31,1000004,5000004,1.0,5.0,5.0
...,...,...,...,...,...
2017-12-31,1029395,5790453,0.0,0.0,0.0
2017-12-31,1029396,5790454,0.0,0.0,0.0
2017-12-31,1029397,5790455,3.0,15.0,15.0


,month_invoiced,customer_id,invoice_id,subscription_type,amount,ongoing_total
0,2014-01-31,1000000,5000000,0.0,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0,5.0
...,...,...,...,...,...,...
790453,2017-12-31,1029395,5790453,0.0,0.0,0.0
790454,2017-12-31,1029396,5790454,0.0,0.0,0.0
790455,2017-12-31,1029397,5790455,3.0,15.0,15.0
790456,2017-12-31,1029398,5790456,0.0,0.0,0.0


In [11]:
# Split the data into train, validate and test (50%, 30%, 20%)
train, validate, test = time_split(df)
train.shape, validate.shape, test.shape

((395229, 6), (237137, 6), (158092, 6))